# Data modelling on neo4j


Import library provided by neo4j for python.

In [ ]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import csv


Connect python notebook to the neo4j database and assign to session the function to run the queries.

In [ ]:
driver = GraphDatabase.driver(uri='bolt://localhost:7687', auth=('neo4j','simone'))
session = driver.session()

# Constraints

For each entity, a uniqueness constraint has been applied on the key prorperty for the main property so that within the DBMS no two nodes of the same type having the same value for the specific property can be entered.

In [ ]:
v1='''CREATE CONSTRAINT ON (c:Characters) ASSERT c.name_character IS UNIQUE'''
#session.run(v1)

In [ ]:
v2='''CREATE CONSTRAINT ON (n:Crews) ASSERT n.name_crew IS UNIQUE'''
#session.run(v2)

In [ ]:
v3='''CREATE CONSTRAINT ON (e:Episode) ASSERT e.number_ep IS UNIQUE'''
#session.run(v3)

In [ ]:
v4 ='''CREATE CONSTRAINT ON (f:Fruit_class) ASSERT f.class_name IS UNIQUE'''
#session.run(v4)

In [ ]:
v5 ='''CREATE CONSTRAINT ON (m:Manga) ASSERT m.number_chapter IS UNIQUE'''
#session.run(v5)

In [ ]:
v6 ='''CREATE CONSTRAINT ON (r:Race) ASSERT r.name_race IS UNIQUE'''
#session.run(v6)

In [ ]:
v7 ='''CREATE CONSTRAINT ON (s:Saga) ASSERT s.name_saga IS UNIQUE'''
#session.run(v7)

In [ ]:
v8 ='''CREATE CONSTRAINT ON (a:Sea) ASSERT a.sea_name IS UNIQUE'''
#session.run(v8)

# Nodes and Properties

Create entities following the previously created a priori schema,all datasets were unified according to neo4j rules in the pre-processing phase and for each property the data type was specified (int,string,float,bool)

**Episode**: describes all the anime episodes up to episode 958.

In [ ]:
q1='''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Anime_neo4j' AS line CREATE (e:Episode {title_ep: toString(line.Title_episode),number_ep:toInteger(line.episode),year_of_release_ep: toInteger(line.start), total_votes_ep:toInteger(line.total_votes), average_rating_ep:toFloat(line.average_rating),
rank_episode_ep:(line.rank_episode)})'''
#session.run(q1)

**Manga**: describes information regarding individual chapters of the Manga.

In [ ]:
q2 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Manga_neo4j' AS line CREATE (m:Manga  {title_chapter:toString(line.Title_chapter), number_chapter:toInteger(line.number_chapter), release_date_chapter:(line.release_date_chapter), page_chapter:toInteger(line.page_chapter)}) '''
#session.run(q2)

**Saga**: describes all the sagas within both the Manga and the Anime.

In [ ]:
q3 ='''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Saghe_neo4j' AS line CREATE(s:Saga {name_saga:toString(line.Arco_narrativo),total_chapters:toInteger(line.Total_Chapters), total_pages:toInteger(line.Total_Pages) ,manga_percentage:(line.Manga_perc),total_episodes:toInteger(line.Total_Episodes),anime_percentage:(line.Anime_perc)}) '''
#session.run(q3)

**Foreshadows**: describes the foreshadowing within the manga and when it is these were revealed.

In [ ]:
q4 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/foreshadows_neo4j' AS line CREATE (f:Foreshadows {setup:toString(line.Setup),payoff:toString(line.Payoff), chapters_later:toInteger(line.Chapters_Later), days_later:toInteger(line.Days_Later), years_later:toInteger(line.Years_Later)}) '''
#session.run(q4)

**Fruit_Class**: describes all the classes that refer to the devil's fruits.

In [ ]:
q5 = '''LOAD CSV with headers FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Class_fruit_neo4j' AS line CREATE (fr:Fruit_class {class_name:toString(line.class)})'''
#session.run(q5)

**Sea**: describes all the seas present in the world of One Piece

In [ ]:
#Creo i nodi dei mari
q6 ='''LOAD CSV with headers FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Sea_neo4j' AS line CREATE (sea:Sea {sea_name:toString(line.sea)})'''
#session.run(q6)

**Race**: describes the races present within the world of One Piece.

In [ ]:
#Creo i nodi delle razze
q7='''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Razze_neo4j' AS line CREATE (r:Race {name_race:toString(line.Race)})'''
#session.run(q7)

**Crews**: describes all the groups and crew in the world of One Piece.

In [ ]:
q8 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Ciurme_neo4j' AS line CREATE (c:Crews {name_crew:toString(line.Nome_ciurma)})'''
#session.run(q8)

**Characters** : describes all character properties updated as of April 2022.

The dataset contains null values so for the creation of these nodes we had to impose constraints on each property so that in case of null value the property is not created for that particular node.

In [ ]:

q9 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_after_PP/Personaggi_neo4j' AS row
MERGE (p:Characters {name_character:(row.Character)})
FOREACH(ignoreMe IN CASE WHEN trim(row.Birth_Month) <> "null" THEN [1] ELSE [] END | SET p.birth_month = toString(row.Birth_Month))
FOREACH(ignoreMe IN CASE WHEN trim(row.Birth_Day) <> "null" THEN [1] ELSE [] END | SET p.birth_day = toString(row.birth_Day))
FOREACH(ignoreMe IN CASE WHEN trim(row.Age) <> "null" THEN [1] ELSE [] END | SET p.age = toInteger(row.Age))
FOREACH(ignoreMe IN CASE WHEN trim(row.deceased) <> "null" THEN [1] ELSE [] END | SET p.deceased = (row.deceased))
FOREACH(ignoreMe IN CASE WHEN trim(row.Birth_Island) <> "null" THEN [1] ELSE [] END | SET p.birth_island = toString(row.Birth_island))
FOREACH(ignoreMe IN CASE WHEN trim(row.Hometown) <> "null" THEN [1] ELSE [] END | SET p.hometown = toString(row.Hometown))
FOREACH(ignoreMe IN CASE WHEN trim(row.Blood_Type) <> "null" THEN [1] ELSE [] END | SET p.bood_type = toString(row.Blood_Type))
FOREACH(ignoreMe IN CASE WHEN trim(row.Height_in_Meters) <> "null" THEN [1] ELSE [] END | SET p.height_in_meters = toFloat(row.Height_in_Meters))
FOREACH(ignoreMe IN CASE WHEN trim(row.Height_in_Foot) <> "null" THEN [1] ELSE [] END | SET p.height_in_foot = toFloat(row.Height_in_Foot))
FOREACH(ignoreMe IN CASE WHEN trim(row.Rank_Height) <> "null" THEN [1] ELSE [] END | SET p.rank_height = toInteger(row.Rank_Height))
FOREACH(ignoreMe IN CASE WHEN trim(row.Bounty_USD) <> "null" THEN [1] ELSE [] END | SET p.bounty_USD = (row.Bounty_USD))
FOREACH(ignoreMe IN CASE WHEN trim(row.Bounty_EUR) <> "null" THEN [1] ELSE [] END | SET p.bounty_EUR = (row.Bounty_EUR))
FOREACH(ignoreMe IN CASE WHEN trim(row.Rank_Wanted) <> "null" THEN [1] ELSE [] END | SET p.rank_wanted = toInteger(row.Rank_Wanted))
FOREACH(ignoreMe IN CASE WHEN trim(row.Devil_Fruit) <> "null" THEN [1] ELSE [] END | SET p.devil_fruit = toString(row.Devil_Fruit))
FOREACH(ignoreMe IN CASE WHEN trim(row.Awakening_fruit_of_devil) <> "null" THEN [1] ELSE [] END | SET p.awakening_fruit_of_devil = (row.Awakening_fruit_of_devil))
FOREACH(ignoreMe IN CASE WHEN trim(row.Armament_Haki) <> "null" THEN [1] ELSE [] END | SET p.armament_haki = (row.Armament_Haki))
FOREACH(ignoreMe IN CASE WHEN trim(row.Observation_Haki) <> "null" THEN [1] ELSE [] END | SET p.observation_haki = (row.Observation_Haki))
FOREACH(ignoreMe IN CASE WHEN trim(row.Conquerors_Haki) <> "null" THEN [1] ELSE [] END | SET p.conquerors_haki = (row.Conquerors_Haki))
FOREACH(ignoreMe IN CASE WHEN trim(row.first_appearance_year) <> "null" THEN [1] ELSE [] END | SET p.first_appearance_year = toInteger(row.first_appearance_year))
'''
#session.run(q9)

# Relations

We import the relationships, using the bridge tables created in pre-processing.

Relation **Saga-[START_ON_EP]->Episode** : links each saga to the anime episode where it begins.

In [ ]:
q10 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_saga_ep_neo4j' AS line
MATCH (s:Saga {name_saga:toString(line.name_saga)}),(e:Episode{number_ep:toInteger(line.start_on_episode)}) 
CREATE(s)-[:START_ON_EP]->(e)'''
#session.run(q10)

Relation **Saga-[START_ON_CHAP]->Manga**: links each saga to the chapter of the manga where it begins.

In [ ]:
q11 ='''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_saga_chap_neo4j' AS line
MATCH (s:Saga {name_saga:toString(line.name_saga)}),(m:Manga{number_chapter:toInteger(line.start_on_chapter)}) 
CREATE(s)-[:START_ON_CHAP]->(m)'''
#session.run(q11)

Relation **Foreshadows-[SETUPPED]->Manga**: links prefigurations to the chapter in which they are introduced.

Relation **Foreshadows-[REVELATE]->Manga**: links prefigurations to the chapter in which they are revealed.

In [ ]:
q12 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_foreshadows_scan__neo4j' AS line
MATCH (f:Foreshadows {setup:toString(line.Setup)}),(m:Manga {number_chapter:toInteger(line.scan_manga)})
CREATE (f)-[:SETUPPED]->(m)'''
#session.run(q12)
q13 = '''LOAD CSV WITH HEADERS FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_foreshadows_scan_revelate_neo4j' AS line
MATCH (f:Foreshadows {payoff:toString(line.Payoff)}),(m:Manga {number_chapter:toInteger(line.scan_manga_rivelato)})
CREATE (f)-[:REVELATE]->(m)'''
#session.run(q13)

Relation **Characters-[FIRST_APPEARANCE_MANGA]->Manga**: links each character to the chapter in which he or she first appears.

In [ ]:
q14 ='''LOAD CSV WITH HEADERS  FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_character_manga_neo4j' AS line
MATCH (c:Characters {name_character:toString(line.Character)}),(m:Manga {number_chapter:toInteger(line.first_appearance_chapter)})
CREATE (c)-[:FIRST_APPEARANCE_MANGA]->(m)'''
#session.run(q14)

Relation **Characters-[FIRST_APPEARANCE_EP]->Episode**: links each character to the episode of the anime in which he or she first appears.

In [ ]:
q15 ='''LOAD CSV WITH HEADERS  FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_character_ep_neo4j' AS line
MATCH (c:Characters {name_character:toString(line.Character)}),(e:Episode {number_ep:toInteger(line.first_appearance_episode)})
CREATE (c)-[:FIRST_APPEARANCE_EP]->(e)'''
#session.run(q15)

Relation **Characters-[WAS_BORN_IN]->Sea**: links each character to the sea in which he or she was born.

In [ ]:
q16 = '''LOAD CSV WITH HEADERS  FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_character_sea_neo4j' AS line
MATCH (c:Characters {name_character:toString(line.Character)}),(sea:Sea {sea_name:toString(line.Birth_sea)})
CREATE (c)-[:WAS_BORN_IN]->(sea)'''
#session.run(q16)

Relation **Characters-[BELONGS_TO_CLASS]->Class_Fruit**: links each of the character's devil fruits to the respective class of the considered fruit.

In [ ]:
q17=  '''LOAD CSV WITH HEADERS  FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_fruit_class_neo4j' AS line
MATCH (c:Characters {name_character:toString(line.Character)}),(n:Fruit_class {class_name:toString(line.Devil_Fruit_Class)})
CREATE (c)-[:BELONGS_TO_CLASS]->(n)'''
#session.run(q17)

Relation **Characters-[BELONGS_TO]->Crews**: connects each character to the crew to which they belong


In [ ]:
q18=  '''LOAD CSV WITH HEADERS  FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_character_crew_neo4j' AS line
MATCH (c:Characters{name_character:toString(line.Character)}),(n:Crews {name_crew:toString(line.Nome_ciurma)})
CREATE (c)-[:BELONGS_TO]->(n)'''
#session.run(q18)

Relation **Characters-[BELONGS_TO_RACE]->Race**: links each character to the race to which they belong.

In [ ]:
q19 =  '''LOAD CSV WITH HEADERS  FROM 'https://github.com/SimoneFarallo/One_Piece_Neo4j_Graph/raw/main/df_bridge_tables/rel_charcater_race_neo4j' AS line
MATCH (c:Characters{name_character:toString(line.Character)}),(n:Race {name_race:toString(line.Race)})
CREATE (c)-[:BELONGS_TO_RACE]->(n)'''
#session.run(q19)

# Query for Data Exploration

Queries were formulated to check that the database was working properly and that the data entered were correct.

**What are the characters born in the east-blue?**

In [ ]:
q20 ='''MATCH (p:Characters)-[:WAS_BORN_IN]->(sea:Sea{sea_name:'East Blue'}) 
RETURN p.name_character'''
#session.run(q20)

**What characters appeared in the first episode who are part of Straw Hat's crew?**

In [ ]:
q21 ='''MATCH (e:Episode{number_ep:'1'})<-[:FIRST_APPEARANCE_EP]-(p:Characters)-[:BELONGS_TO]->(c:Crews{name_crew:'Pirati di Cappello di paglia'}) 
RETURN p.name_character'''
#session.run(q21)

**What are the characters belonging to Big Mom's crew who possess the devil fruit? What class does their fruit belong to?**

In [ ]:
q22 ='''MATCH(f:Fruit_class)<-[:BELONGS_TO_CLASS]- (c:Characters)-[:BELONGS_TO]->(n:Crews{name_crew:'Pirati di Big Mom'})  
WHERE c.devil_fruit is not  null
RETURN c.name_character,c.devil_fruit,f.class_name'''
#session.run(q22)

**Find all East Blue-born characters belonging to the human race and sort them according to the wanted ranking**

In [ ]:
q23 = '''MATCH (s:Sea{sea_name:'East Blue'})<-[:WAS_BORN_IN]-(p:Characters)-[:BELONGS_TO_RACE]->(r:Race{name_race:'Human'}) 
WHERE p.bounty_EUR is not null             
RETURN p.name_character,p.bounty_EUR,p.rank_wanted'''
#session.run(q23)

**Which characters appeared first in the episode with the highest grade point average?**

In [ ]:
q24 ='''MATCH (e:Episode)<-[:FIRST_APPEARANCE_EP]-(c:Characters) 
WITH max(e.average_rating_ep) AS max
MATCH (e:Episode)<-[:FIRST_APPEARANCE_EP]-(c:Characters) 
WHERE e.average_rating_ep=max 
RETURN e,c'''
#session.run(q24)

**Which saga has the most episodes and in which episode does it begin?**

In [ ]:
q25 = '''MATCH (n:Saga)-[:START_ON_EP]->(e:Episode) 
WITH max(n.total_episodes) AS max 
MATCH  (n:Saga)-[:START_ON_EP]->(e:Episode) 
WHERE n.total_episodes=max  
RETURN n,e'''
#session.run(q25)

**Which characters belonging to the Hundred Beasts crew are in possession of a sea fruit? Sort them according to the wanted ranking**

In [ ]:
q26 = '''MATCH (n:Characters)-[:BELONGS_TO]->(c:Crews{name_crew:'Pirati delle cento bestie'}) 
WHERE n.devil_fruit is not null and n.rank_wanted is not null 
RETURN n.name_character,n.devil_fruit,n.rank_wanted,n.bounty_EUR 
ORDER BY n.rank_wanted'''
#session.run(q26)

**Who is the shortest character in One Piece? What race and what crew do you belong to?**

In [ ]:
q27 = '''MATCH (n:Crews)<-[:BELONGS_TO]- (c:Characters)-[:BELONGS_TO_RACE]->(r:Race) 
WITH min(c.height_in_meters) AS min 
MATCH (n:Crews)<-[:BELONGS_TO]- (c:Characters)-[:BELONGS_TO_RACE]->(r:Race) 
WHERE c.height_in_meters = min 
RETURN c,n,r'''
#session.run(q27)

**When the foreshadowing regarding "Monkey D. Dragon brings a wounded person to the ship" is revealed and what is revealed?**

In [ ]:
q28 ='''MATCH (f:Foreshadows)-[r:SETUPPED]->(m:Manga) 
WHERE f.setup='Monkey D. Dragon brings a wounded person to the ship' 
RETURN f,m'''
#session.run(q28)

**When did the saga in the Anime of "Syrup Village Arc" begin? Are there any characters belonging to any crew or group?**

In [ ]:
q29 ='''MATCH (s:Saga{name_saga:'Syrup Village Arc'})-[:START_ON_EP]->(e:Episode)<-[:FIRST_APPEARANCE_EP]-(c:Characters)-[:BELONGS_TO]->(a:Crews) 
RETURN c.name_character,a.name_crew,e.number_ep,e.title_ep'''
#session.run(q29)

**Who are the characters who belong to the human race,are part of the group "Pirate Heart" and are in possession of a devil fruit?**

In [ ]:
q30 = '''MATCH  (r:Race{name_race:'Human'})<-[:BELONGS_TO_RACE]-(c:Characters)-[:BELONGS_TO]->(a:Crews) 
WHERE a.name_crew='Pirati Heart' AND c.devil_fruit IS NOT NULL 
RETURN c'''
#session.run(q30)

**Chi sono i personaggi più alt**Who are the characters taller than 10 feet? What race do they belong to?**i di 3 metri? A che razza appartengono?**

In [ ]:
q31 = '''MATCH (n:Characters)-[:BELONGS_TO_RACE]->(c:Race)
WHERE n.height_in_meters > 3
RETURN n,c'''
#session.run(q31)


After downloading the Neo4j plugin extension called APOC, the following commands were used to export the database:

In [ ]:
q32='''CALL apoc.export.json.all("one_piece_graph(JSON)",{useTypes:true})'''
#session.run(32)
q33='''CALL apoc.export.csv.all("one_piece_graph(CSV)", {})'''
#session.run(33)


The database was saved in both CSV and JSON formats.